In [28]:
import pandas as pd
import json

In [3]:
df = pd.read_csv('all_seasons.csv', index_col = 0)

# Creating node-link data

In [17]:
grouped = df.groupby(['player_name', 'team_abbreviation']).agg({'season': 'unique'}).reset_index()

In [18]:
player_ids = {}


nodes = []
for i, player in enumerate(grouped['player_name'].unique()):
    nodes.append({
        'id': i+1,
        'name': player
    })
    player_ids[player] = i+1


In [29]:
player_team_seasons = {}
for _, row in grouped.iterrows():
    player = row['player_name']
    team = row['team_abbreviation']
    seasons = row['season']
    player_team_seasons[(player, team)] = seasons


links = []

#for each player, team, season combo
for (player1, team1), seasons1 in player_team_seasons.items():
    for (player2, team2), seasons2 in player_team_seasons.items():
        if player1 != player2 and team1 == team2:
            common_seasons = set(seasons1).intersection(seasons2)
            if len(common_seasons) > 0:
                links.append({
                    'source': player_ids[player1],
                    'target': player_ids[player2],
                    'value': len(common_seasons)
                })


json_data = {
    'nodes': nodes,
    'links': links
}

with open('player_connections.json', 'w') as json_file:
    json.dump(json_data, json_file)